# Project B: Supply Chain Data Cleaning Pipeline 🚛🧹
**Author:** Kilian Sender
**Status:** ✅ Completed (v1.0)

## 1. Setup & Data Ingestion

### Data Source
The dataset **"Supply Chain Shipment Pricing Data"** was sourced from **Pushpit Kamboj**.
* **Original Link:** https://www.kaggle.com/datasets/pushpitkamboj/logistics-data-containing-real-world-data
* **License:** CC0: Public Domain

Loading necessary libraries and the raw CSV file.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Dataset
# We save it into 'df_raw' to always have a backup of the original state
df_raw = pd.read_csv('raw_supply_chain_data.csv', encoding='ISO-8859-1')

# Display first rows
df_raw.head()

,payment_type,profit_per_order,sales_per_customer,category_id,category_name,customer_city,customer_country,customer_id,customer_segment,customer_state,...,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,shipping_date,shipping_mode,label
0,DEBIT,34.448338,92.49099,9.0,Cardio Equipment,Caguas,Puerto Rico,12097.6830,Consumer,PR,...,Western Europe,Vienna,COMPLETE,191.0,9.0,Nike Men's Free 5.0+ Running Shoe,99.99,2015-08-13 00:00:00+01:00,Standard Class,-1
1,TRANSFER,91.193540,181.99008,48.0,Water Sports,Albuquerque,EE. UU.,5108.1045,Consumer,CA,...,South America,Buenos Aires,PENDING,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2017-04-09 00:00:00+01:00,Standard Class,-1
2,DEBIT,8.313806,89.96643,46.0,Indoor/Outdoor Games,Amarillo,Puerto Rico,4293.4478,Consumer,PR,...,Western Europe,Nord-Pas-de-Calais-Picardy,COMPLETE,1014.0,46.0,O'Brien Men's Neoprene Life Vest,49.98,2015-03-18 00:00:00+00:00,Second Class,1
3,TRANSFER,-89.463196,99.15065,17.0,Cleats,Caguas,Puerto Rico,546.5306,Consumer,PR,...,Central America,Santa Ana,PROCESSING,365.0,17.0,Perfect Fitness Perfect Rip Deck,59.99,2017-03-18 00:00:00+00:00,Second Class,0
4,DEBIT,44.722590,170.97824,48.0,Water Sports,Peabody,EE. UU.,1546.3980,Consumer,CA,...,Central America,Illinois,COMPLETE,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2015-03-30 00:00:00+01:00,Standard Class,1


## 2. Initial Inspection (Exploratory Data Analysis)
We analyze the dataframe structure, data types, and check for missing values.

In [2]:
df_raw.head()

,payment_type,profit_per_order,sales_per_customer,category_id,category_name,customer_city,customer_country,customer_id,customer_segment,customer_state,...,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,shipping_date,shipping_mode,label
0,DEBIT,34.448338,92.49099,9.0,Cardio Equipment,Caguas,Puerto Rico,12097.6830,Consumer,PR,...,Western Europe,Vienna,COMPLETE,191.0,9.0,Nike Men's Free 5.0+ Running Shoe,99.99,2015-08-13 00:00:00+01:00,Standard Class,-1
1,TRANSFER,91.193540,181.99008,48.0,Water Sports,Albuquerque,EE. UU.,5108.1045,Consumer,CA,...,South America,Buenos Aires,PENDING,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2017-04-09 00:00:00+01:00,Standard Class,-1
2,DEBIT,8.313806,89.96643,46.0,Indoor/Outdoor Games,Amarillo,Puerto Rico,4293.4478,Consumer,PR,...,Western Europe,Nord-Pas-de-Calais-Picardy,COMPLETE,1014.0,46.0,O'Brien Men's Neoprene Life Vest,49.98,2015-03-18 00:00:00+00:00,Second Class,1
3,TRANSFER,-89.463196,99.15065,17.0,Cleats,Caguas,Puerto Rico,546.5306,Consumer,PR,...,Central America,Santa Ana,PROCESSING,365.0,17.0,Perfect Fitness Perfect Rip Deck,59.99,2017-03-18 00:00:00+00:00,Second Class,0
4,DEBIT,44.722590,170.97824,48.0,Water Sports,Peabody,EE. UU.,1546.3980,Consumer,CA,...,Central America,Illinois,COMPLETE,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2015-03-30 00:00:00+01:00,Standard Class,1


## 3. Data Cleaning
### Issue 1: Corrupted IDs (Float to Int)
**Problem:** IDs (Customer, Order, Product) are interpreted as floats (e.g., `12097.0`).
**Fix:** Cast columns to standard Integer types to ensure referential integrity.

### Issue 2: Zip Code Formatting
**Problem:** Zip codes contain decimals (`725.0`) and are missing leading zeros.
**Fix:** Convert to string, remove decimals, and pad with zeros to standard 5-digit format.

### Issue 3: Date & Time Parsing
**Problem:** `order_date` and `shipping_date` are stored as object strings.
**Fix:** Parse columns to datetime objects for time-series analysis.



In [3]:
# Create a working copy to preserve raw data
df = df_raw.copy()

# --- FIX: ZIP CODES ---
# First convert to 'int' (truncates decimals), then to 'str'.
# This is necessary because 'int' prevents decimals, but we need 'str' to allow leading zeros.
df['Zip_Clean'] = pd.to_numeric(df['customer_zipcode'], errors='coerce').fillna(0).astype(int).astype(str)

# Pad with leading zeros to ensure a standard length of 5 digits
df['Zip_Clean'] = df['Zip_Clean'].str.zfill(5)


# --- FIX: MONETARY COLUMNS (ROUNDING) ---
# Prices technically cannot have more than 2 decimal places.
# We remove "noise" (e.g., floating point errors like 47.9397 -> 47.94).

money_cols = [
    'product_price',
    'sales',
    'order_item_total_amount',
    'profit_per_order',
    'sales_per_customer',
    'order_profit_per_order',
    'order_item_discount',
    'order_item_product_price',
    ]

# Pro-Tip: Batch process all monetary columns at once
for col in money_cols:
    # Check if column exists (failsafe against typos)
    if col in df.columns:
        df[col] = df[col].round(2)

# Verification
print("Prices after cleaning:")
print(df[money_cols].head())


# --- FIX: PERCENTAGE VALUES (RATES & RATIOS) ---
# Here too, we round to 2 decimal places (0.1234 -> 0.12)
rate_cols = ['order_item_discount_rate', 'order_item_profit_ratio']

for col in rate_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').round(2)


# --- FIX: INTEGERS (IDs & QUANTITIES) ---
# These columns must strictly not have decimals. We perform a radical cleanup.

# List of columns that MUST be Integers
cols_to_int = [
    'order_item_quantity',
    'order_item_id',
    'order_item_cardprod_id',
    'order_id',
    'order_customer_id',
    'department_id', 
    'category_id',
    'department_id',
    'customer_id'
    ]

for col in cols_to_int:
    # 1. Convert to numeric first (to catch/coerce errors)
    # 2. Fill NaNs with 0 (prevents crash during astype(int))
    # 3. Hard cast to Integer
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)


# --- VERIFICATION ---
print("New Data Types:")
print(df[cols_to_int].dtypes)
print("\nSample Data:")
print(df[cols_to_int].head(3))


# --- FIX: TEXT (STRINGS) ---
# Removes leading/trailing whitespace from ALL text columns
# select_dtypes(include='object') automatically grabs all string columns
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())


# --- FIX: PRODUCT IDs ---
# There is "noise" (decimals) here as well that needs to be truncated.
cols_to_fix = ['product_card_id', 'product_category_id']

for col in cols_to_fix:
    # 1. Fillna(0) as safety net if NaNs exist
    # 2. astype(int) truncates the decimal
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)


# --- FIX: DATE FORMATS ---

# Conversion to DateTime objects
# pandas automatically attempts to infer the correct format
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce', utc=True)
df['shipping_date'] = pd.to_datetime(df['shipping_date'], errors='coerce', utc=True)

# Final Verification
df[['Zip_Clean','order_date', 'shipping_date','product_card_id', 'product_category_id']].head()
print("Cleaning pipeline completed successfully.")

Prices after cleaning:
   product_price   sales  order_item_total_amount  profit_per_order  \
0          99.99   99.99                    84.99             34.45   
1         199.99  199.99                   181.99             91.19   
2          49.98   99.96                    93.81              8.31   
3          59.99  119.98                    99.89            -89.46   
4         199.99  199.99                   171.08             44.72   

   sales_per_customer  order_profit_per_order  order_item_discount  \
0               92.49                   32.08                12.62   
1              181.99                   91.24                16.50   
2               89.97                    6.97                 6.60   
3               99.15                  -95.40                16.94   
4              170.98                   44.57                29.99   

   order_item_product_price  
0                     99.99  
1                    199.99  
2                     49.98  
3        

In [4]:
# Sanity Check: Validate the transformation (Raw vs. Cleaned)
df[['customer_zipcode', 'Zip_Clean']].head()

,customer_zipcode,Zip_Clean
0,725.0000,00725
1,92745.1600,92745
2,2457.7297,02457
3,725.0000,00725
4,95118.6000,95118


**Decision:** The `customer_zipcode` column contains significant **synthetic noise** (decimal artifacts) and data inconsistencies.
Therefore, it will be excluded from the core analysis. We will instead rely on `order_country` for reliable geographical aggregation.

## 4. Logic Validation & Feature Engineering
We calculate the `actual_shipping_days` and validate the data against physical constraints.

### ⚠️ Anomaly Detection: "Time Travel"
The analysis of shipping durations reveals records with **negative values** (Shipping Date precedes Order Date).
* **Observation:** Approx. 40% of the dataset is affected. Extreme outlier: -1,429 days.
* **Action:** Removal of these rows to ensure statistical integrity for Lead Time Analysis.

In [5]:
# Calculation: Shipping Date - Order Date
df['actual_shipping_days'] = (df['shipping_date'] - df['order_date']).dt.days

# PLAUSIBILITY CHECK:
# Are there negative shipping durations? (Shipment BEFORE Order?)
negative_days = df[df['actual_shipping_days'] < 0]

print("--- TIME TRAVEL ANALYSIS ---")
print(f"Count of illogical records (negative): {len(negative_days)}")
print("\nError Examples:")
print(negative_days[['order_date', 'shipping_date', 'actual_shipping_days']].head(3))

# Error Visualization (Optional but insightful)
print("\nError Statistics:")
print(negative_days['actual_shipping_days'].describe())

--- TIME TRAVEL ANALYSIS ---
Count of illogical records (negative): 5941

Error Examples:
                 order_date             shipping_date  actual_shipping_days
3 2017-05-30 23:00:00+00:00 2017-03-18 00:00:00+00:00                   -74
7 2016-06-08 23:00:00+00:00 2016-04-23 23:00:00+00:00                   -46
9 2017-08-28 23:00:00+00:00 2017-04-27 23:00:00+00:00                  -123

Error Statistics:
count    5941.000000
mean      -72.394041
std        87.001059
min     -1429.000000
25%      -100.000000
50%       -40.000000
75%       -15.000000
max        -1.000000
Name: actual_shipping_days, dtype: float64


### ⚠️ Data Anomaly Detected!
Analysis reveals 5,941 records with **negative shipping durations** (reaching up to -1,429 days).
This is physically impossible (shipment occurring ~4 years *before* the order).

**Root Cause:** Likely data entry errors or artifacts from synthetic data generation.
**Decision:** Since time travel has not been invented yet, these rows are classified as "Data Errors" and are **removed** to prevent skewing the statistical mean.

### ⚠️ Additional Anomaly: Zombie Shipments
Besides negative durations, we also detected **extremely unrealistic shipping times**.

**Extreme Example:** An order to "Nigeria, Southern Europe" showing a delivery time of **1,430 days**.

**Action:** We apply a **hard cap of 120 days**. Any record exceeding this threshold is considered a data error and is removed to ensure data quality.

In [6]:
# CLEANUP PHASE
# We retain only rows where the shipping duration is non-negative (>= 0)
rows_before = len(df)
df = df[df['actual_shipping_days'] >= 0]
rows_after = len(df)

print(f"Cleanup completed.")
print(f"Dropped rows (Negative Duration): {rows_before - rows_after}")
print(f"Remaining dataset: {rows_after}")

# --- UPPER BOUND CHECK ---
# We observed shipping times > 1400 days ("Zombie Shipments").
# Realistic Assumption: Anything above 120 days (4 months) is considered a data error or lost cargo.

rows_before_upper = len(df)
df = df[df['actual_shipping_days'] <= 120]  # Cutoff at 120 days
rows_after_upper = len(df)

print(f"Extreme outliers (>120 days) removed: {rows_before_upper - rows_after_upper}")

Cleanup completed.
Dropped rows (Negative Duration): 5941
Remaining dataset: 9608
Extreme outliers (>120 days) removed: 1246


**Decision:** Due to massive inconsistencies (~40% of records violate **temporal causality**), we prioritize **Quality over Quantity**.

These records are removed to safeguard the integrity of downstream analyses (e.g., Lead Time Prediction).

## 5. Final Cleanup & Optimization
Now that the new, sanitized columns (e.g., `Zip_Clean`) have been successfully generated, we **drop the obsolete raw columns**.
Finally, we validate the data types to ensure schema consistency.

**Goal:** A streamlined, analysis-ready dataset free of redundancies.

In [7]:
# --- FINAL SCHEMA OPTIMIZATION ---
# Since the validation of 'Zip_Clean' was successful, we swap it in.
# We drop the obsolete raw column and rename the clean version to the standard name.

df = df.drop(columns=['customer_zipcode'])
df = df.rename(columns={'Zip_Clean': 'customer_zipcode'})

print("✅ Obsolete columns dropped. Dataset is clean and normalized.")
print(f"Remaining Float Columns: {df.select_dtypes(include=['float']).columns.tolist()}")

✅ Obsolete columns dropped. Dataset is clean and normalized.
Remaining Float Columns: ['profit_per_order', 'sales_per_customer', 'latitude', 'longitude', 'order_item_discount', 'order_item_discount_rate', 'order_item_product_price', 'order_item_profit_ratio', 'sales', 'order_item_total_amount', 'order_profit_per_order', 'product_price']


## 6. Data Export & Persistence
The cleaned dataset is exported to CSV. It serves as the **Golden Record** for downstream analytics and the Streamlit Dashboard.

In [8]:
# EXPORT TO CSV
# Save without index (row numbers are redundant in the CSV)
output_filename = 'cleaned_supply_chain_data.csv'
df.to_csv(output_filename, index=False)

print(f"✅ File successfully saved as: {output_filename}")
print(f"Dimensions: {df.shape[0]} rows, {df.shape[1]} columns")

✅ File successfully saved as: cleaned_supply_chain_data.csv
Dimensions: 8362 rows, 42 columns
